## Partie 2 : Équation non linéaire
Dans cette section, nous appliquons des méthodes de résolution d’équations non linéaires afin d’analyser divers indicateurs épidémiologiques.

In [1]:
#Importation des bibliotheque necessaires
import pandas as pd
import numpy as np

# Importation des modules internes du projet
from src.analysis import estimer_parametres_rab
from src.analysis.equations_differentielles.simulateur_sird import SimulateurSIRD
from src.analysis.integration.methodes import Integration
from src.data import DataPipeline

In [2]:
# Chargement des données pour le pays France
pipeline = DataPipeline(country="France")
df = pipeline.run(smoothing=True)

population = pipeline.population
parametres = estimer_parametres_rab(df)

In [3]:
# Affectation de chaque variable avec sa valeur
t = df.index.values
S = df["S"].values
I = df["I"].values
Imax=df["lits_par_mille"].iloc[0]/1000 
r=estimer_parametres_rab(df)['r']
a=estimer_parametres_rab(df)['a']
b=estimer_parametres_rab(df)['b']

### Question a) Détermination du pic épidémique  
**Objectif** : Trouver le moment où l'épidémie atteint son pic (nombre d'infectés maximal).  
**Méthode** : Résoudre l'équation non linéaire:  $rS(t)I(t) - (a + b)I(t) = 0 \Rightarrow S = \frac{a + b}{r}$

In [4]:
#calcul du pic epedimique
dI_dt = r * I * S - (a + b) * I
# Recherche du changement de signe de la dérivée dI/dt, 
# ce qui indique un changement de monotonicité de I(t) 
# (passage d'une phase croissante à décroissante, donc un pic épidémique)
changements_signe = np.where(np.diff(np.sign(dI_dt)) < 0)[0]

#Verification s'il y a un changement de signe ou non
if len(changements_signe) == 0:
    pic_epidemique=None
else:
    index_pic_epidemique=t[changements_signe[0] + 1]  # +1 car diff réduit la longueur
    pic_epidemique=S[changements_signe[0]+1] #+1 car diff reduit la longuer

#### Résultats

In [5]:
print(f"La valeur de S et t au pic epedimique est:","\n",f"S={pic_epidemique}","\n",f"t={index_pic_epidemique}")

La valeur de S et t au pic epedimique est: 
 S=0.9999997640570392 
 t=69


### Question b) Seuil d'immunité collective (HIT)  
**Objectif** : Calculer la proportion minimale de la population immunisée pour arrêter l'épidémie.  
**Méthode** : Le seuil critique est donné par : $P_c = 1 - S_c = 1 - \frac{a + b}{r}.$

In [6]:
#Seuil critique des personnes susceptible
Sc=(a+b)/r
#la proportion minimale de la population qui doit être immunisée
Pc=1-Sc


#### Résultats

In [7]:
print(f"la proportion minimale de la population qui doit être immunisée est : Pc={Pc}")

la proportion minimale de la population qui doit être immunisée est : Pc=0.26334517263093205


### Question c) Temps critique pour une intervention sanitaire  
**Objectif** : Identifier le moment où les infections dépassent la capacité hospitalière $I_{\text{max}}$.  
**Méthode** : Résoudre numériquement l'équation : $I(t_c) = I_{\text{max}}$

In [8]:
#estimation de tc: temps critique
tc=None
for i, val in enumerate(I):
    if val > Imax:
        tc=i 
        break
    

#### Résultats

In [9]:
print(f"Le temps critique est: tc={tc}")

Le temps critique est: tc=299


### Question d) Calcul du nombre de reproduction de base $R_0$  
**Objectif** : Estimer $R_0$ à partir des paramètres épidémiques.  
**Méthode** : Utiliser la formule théorique :  $R_0 = \frac{r}{a + b}$

In [10]:
#Calcul du nombre de reproduction de base R_0
R0=(r)/(a+b)

#### Résultats

In [11]:
print(f"Le nombre de reproduction de base R_0 est:\nR0={R0}")

Le nombre de reproduction de base R_0 est:
R0=1.3574878801398185
